In [0]:
%sql
CREATE CATALOG IF NOT EXISTS casino_ctg;
USE catalog casino_ctg;
CREATE SCHEMA IF NOT EXISTS log_db;
USE schema log_db;
CREATE VOLUME IF NOT EXISTS sourcefiles;
TRUNCATE TABLE IF EXISTS transactions_bronze; 

In [0]:
# Define the target location in a Unity Catalog Volume
volume_path = "dbfs:/Volumes/casino_ctg/log_db/sourcefiles"
dbutils.fs.mkdirs(volume_path)

In [0]:
import pyspark.pandas as ps
import re
from pyspark.sql.functions import col

# --- CONFIGURATION ---
# Base path where all the raw CSVs were downloaded
BASE_VOLUME_PATH = "/Volumes/casino_ctg/log_db/sourcefiles"
TARGET_CATALOG = "casino_ctg"
TARGET_SCHEMA = "log_db"
DATE_COLUMN = 'date'
TS_COLUMN ='timestamp'
# 1. List all CSV files in the raw_files directory
try:
    # dbutils.fs.ls returns a list of FileInfo objects
    file_list = dbutils.fs.ls(BASE_VOLUME_PATH)
   # print(file_list)
    trans_list =[ll for ll in file_list if ll.name.startswith('transactions')]
except Exception as e:
    print(f"Error listing files in {BASE_VOLUME_PATH}: {e}")
    file_list = []



In [0]:
filename = 'games_metadata.csv'
table_name = f"games_bronze"
full_table_name = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{table_name}"
raw_csv_path = f"{BASE_VOLUME_PATH}/{filename}"

try:
    # Load data using Spark DataFrame API to avoid /dbfs path access issues
    df_spark = spark.read.option("header", True).csv(raw_csv_path)
    
    # Check for the existence of the 'date' column and perform conversion if present
    if DATE_COLUMN in df_spark.columns:
        print(f"  Transforming column '{DATE_COLUMN}' to datetime type.")
        df_spark = df_spark.withColumn(
            DATE_COLUMN,
            col(DATE_COLUMN).cast("timestamp")
        )
    else:
        print(f"  Column '{DATE_COLUMN}' not found, skipping datetime transformation.")

    if TS_COLUMN in df_spark.columns:
        print(f"  Transforming column '{TS_COLUMN}' to datetime type.")
        df_spark = df_spark.withColumn(
            TS_COLUMN,
            col(TS_COLUMN).cast("timestamp")
        )
    else:
        print(f"  Column '{TS_COLUMN}' not found, skipping datetime transformation.")

    # Write to the Bronze Delta Table using the UC naming convention
    df_spark.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
    
    print(f"  Successfully wrote {df_spark.count()} rows to {full_table_name}.")

except Exception as e:
    print(f"  FAILED to process {filename}. Error: {e}")

  Column 'date' not found, skipping datetime transformation.
  Column 'timestamp' not found, skipping datetime transformation.
  Successfully wrote 4 rows to casino_ctg.log_db.games_bronze.


In [0]:
# 2. Iterate over each file, process, and write to Bronze Delta Table
for file_info in trans_list:
    filename=file_info.name
    table_name = f"transactions_bronze"
    full_table_name = f"{TARGET_CATALOG}.{TARGET_SCHEMA}.{table_name}"
    raw_csv_path = f"{BASE_VOLUME_PATH}/{filename}"
    
    print(f"\nProcessing file: {filename} -> Target table: {full_table_name}")
    
    try:
        # Load data using Spark DataFrame API to avoid /dbfs path access issues
        df_spark = spark.read.option("header", True).csv(raw_csv_path)
        
        # Check for the existence of the 'date' column and perform conversion if present
        if DATE_COLUMN in df_spark.columns:
            print(f"  Transforming column '{DATE_COLUMN}' to datetime type.")
            df_spark = df_spark.withColumn(DATE_COLUMN, col(DATE_COLUMN).cast("timestamp"))
        else:
            print(f"  Column '{DATE_COLUMN}' not found, skipping datetime transformation.")

        if TS_COLUMN in df_spark.columns:
            print(f"  Transforming column '{TS_COLUMN}' to datetime type.")
            df_spark = df_spark.withColumn(TS_COLUMN, col(TS_COLUMN).cast("timestamp"))
        else:
            print(f"  Column '{TS_COLUMN}' not found, skipping datetime transformation.")

        # Write to the Bronze Delta Table using the UC naming convention
        df_spark.write.format("delta").mode("append").saveAsTable(full_table_name)
        
        print(f"  Successfully wrote {df_spark.count()} rows to {full_table_name}.")

    except Exception as e:
        print(f"  FAILED to process {filename}. Error: {e}")